> A project to analyze hacker news stories using nbdbt

## Analysis

In [1]:
#| echo: false
%reload_ext autoreload
%autoreload 2

In [2]:
#| echo: false
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
#| echo: false
%reload_ext nbdbt.dbt_cellmagic

In [4]:
#| echo: false
%dbtconfig -p ../hn_whos_hiring -n notebooks/whos_hiring.ipynb

### How to filter only articles with whos hiring

In [16]:
%%dbt -a whos_hiring_articles analyses/whos_hiring_articles.sql
select *
from {{ ref('hn_posts') }}
where post_type = 'story'
 and  lower(title) like '%ask hn: who is hiring%'
 and  submitter_id = 'whoishiring'
order by submit_timestamp desc


In [17]:
hiring_df = whos_hiring_articles.ref()

In [18]:
hiring_df

,post_id,title,url,submitter_id,content,submit_timestamp,dead,post_score,parent_id,post_type,ranking,deleted,descendants
0,31947297,Ask HN: Who is hiring? (July 2022),None,whoishiring,"Please state the location and include REMOTE, ...",2022-07-01 15:00:14+00:00,False,328,NaN,story,NaN,None,624.0
1,31582796,Ask HN: Who is hiring? (June 2022),None,whoishiring,"Please state the location and include REMOTE, ...",2022-06-01 15:01:51+00:00,False,390,NaN,story,NaN,None,862.0
2,31235968,Ask HN: Who is hiring? (May 2022),None,whoishiring,"Please state the location and include REMOTE, ...",2022-05-02 15:01:38+00:00,False,390,NaN,story,NaN,None,851.0
3,30878761,Ask HN: Who is hiring? (April 2022),None,whoishiring,"Please state the location and include REMOTE, ...",2022-04-01 15:02:05+00:00,False,264,NaN,story,NaN,None,805.0
4,30515750,Ask HN: Who is hiring? (March 2022),None,whoishiring,"Please state the location and include REMOTE, ...",2022-03-01 16:01:56+00:00,False,327,NaN,story,NaN,None,875.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2949787,Ask HN: Who is Hiring? (September 2011),None,whoishiring,Please lead with the location of the position ...,2011-09-01 12:55:49+00:00,False,249,NaN,story,NaN,None,232.0
140,2831646,Ask HN: Who is Hiring? (August 2011),None,whoishiring,Please lead with the location of the position ...,2011-08-01 11:52:07+00:00,False,282,NaN,story,NaN,None,307.0
141,2607052,Ask HN: Who is Hiring? (June 2011),None,whoishiring,Please lead with the location of the position ...,2011-06-01 13:00:55+00:00,False,326,NaN,story,NaN,None,329.0
142,2503204,Ask HN: Who is Hiring? (May 2011),None,whoishiring,Please lead with the location of the position ...,2011-05-01 15:54:54+00:00,False,324,NaN,story,NaN,None,291.0


In [20]:
%%dbt -a whos_hiring  models/whos_hiring_articles.sql
select *
from {{ ref('hn_posts') }}
where post_type = 'story'
 and  lower(title) like '%ask hn: who is hiring%'
 and  submitter_id = 'whoishiring'


In [21]:
# %cd ../hn_whos_hiring
# !dbt run -s models/whos_hiring_articles.sql
# %cd ../notebooks

/home/butch2/play/experiments/hn_whos_hiring/notebooks
17:00:10  Running with dbt=1.1.1
17:00:10  Found 2 models, 0 tests, 0 snapshots, 14 analyses, 191 macros, 0 operations, 0 seed files, 1 source, 0 exposures, 0 metrics
17:00:10  
17:00:12  Concurrency: 1 threads (target='dev')
17:00:12  
17:00:12  1 of 1 START view model 00dev.whos_hiring_articles ............................. [RUN]
17:00:13  1 of 1 OK created view model 00dev.whos_hiring_articles ........................ [OK in 1.56s]
17:00:13  
17:00:13  Finished running 1 view model in 3.00s.
17:00:13  
17:00:13  Completed successfully
17:00:13  
17:00:13  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1
/home/butch2/play/experiments/hn_whos_hiring/notebooks


In [25]:
%%dbt -a job_posts analyses/job_posts.sql
with hiring as
( select post_id,
  from {{ ref('whos_hiring_articles') }}
),
job_posts as 
(
  select p.*
  from {{ ref('hn_posts') }} p
  inner join hiring h
  on p.parent_id = h.post_id
)
select *  
from job_posts
limit 50

In [26]:
jobs_df = job_posts.ref()

In [27]:
jobs_df

,post_id,title,url,submitter_id,content,submit_timestamp,dead,post_score,parent_id,post_type,ranking,deleted,descendants
0,14902492,None,None,cybus,"Cybus.io | Hamburg, Germany | Node.JS Develope...",2017-08-01 16:30:43+00:00,False,NaN,14901313,comment,NaN,None,NaN
1,14902493,None,None,nickdavidhaynes,"Automated Insights | Software engineer, applie...",2017-08-01 16:30:52+00:00,False,NaN,14901313,comment,NaN,None,NaN
2,14025779,None,None,bpp198,Thread.com (YC S12) | Software Engineer | Lond...,2017-04-03 18:44:18+00:00,False,NaN,14023198,comment,NaN,None,NaN
3,25645527,None,None,sdalezman,Intello | Senior Software Engineer &#x2F; Prod...,2021-01-05 13:57:39+00:00,False,NaN,25632982,comment,NaN,None,NaN
4,11497114,None,None,morenoh149,"CapsuleRx | New York City, NY NYC | full-time ...",2016-04-14 14:45:21+00:00,False,NaN,11405239,comment,NaN,None,NaN
5,21136425,None,None,arcfenix,Xfers.com |Backend Engineer (team lead) | Full...,2019-10-02 13:54:24+00:00,False,NaN,21126014,comment,NaN,None,NaN
6,28386447,None,None,claireatshelf,"Shelf Engine | (<a href=""https:&#x2F;&#x2F;www...",2021-09-01 22:44:20+00:00,False,NaN,28380661,comment,NaN,None,NaN
7,8687784,None,None,mazerj,Location: Culver City (Los Angeles)\nPosition:...,2014-12-02 16:11:59+00:00,False,NaN,8681040,comment,NaN,None,NaN
8,8687789,None,None,summerville,Frontend Software Engineer @ Fleetio - <a href...,2014-12-02 16:13:18+00:00,False,NaN,8681040,comment,NaN,None,NaN
9,6653441,None,None,ejdyksen,"Mutually Human - Grand Rapids, Michigan<p><a h...",2013-11-01 13:18:46+00:00,False,NaN,6653437,comment,NaN,None,NaN


In [28]:
jobs_df.iloc[0].content

'Cybus.io | Hamburg, Germany | Node.JS Developer (m&#x2F;f) | Full time | ONSITE\nCybus is looking for a motivated Senior Node.js Developer (m&#x2F;f). You will build a robust, modular system to deliver flexible solutions to our customers. Drive our middleware forward in terms of scalability and reliability and ensure that our system stays cutting-edge while keeping high quality standards.\nYou ideally if you enjoy building awesome applications from scratch. You&#x27;re excellent knowledge of best practices in JavaScript and a deep understanding of Test Driven Development &amp; Clean Code.\nWhat We offer \n- a young &amp; motivated tech company.\n- startup opportunities, atmosphere and latest industry gadgets\n- “Family first” policy – flexible working hours and spare time\n- fresh fruits, juice and fantastic coffee.\n- regular team events and relaxed after-work activities.\n- attractive additional services\n<a href="https:&#x2F;&#x2F;www.cybus.io&#x2F;de&#x2F;tech-jobs-de&#x2F;senior-

In [29]:
%%dbt -a job_posts models/job_posts.sql
with hiring as
( select post_id,
  from {{ ref('whos_hiring_articles') }}
),
job_posts as 
(
  select p.*
  from {{ ref('hn_posts') }} p
  inner join hiring h
  on p.parent_id = h.post_id
)
select *  
from job_posts

In [30]:
# %cd ../hn_whos_hiring
# !dbt run -s models/job_posts.sql
# %cd ../notebooks

/home/butch2/play/experiments/hn_whos_hiring/notebooks
17:13:20  Running with dbt=1.1.1
17:13:20  Found 3 models, 0 tests, 0 snapshots, 15 analyses, 191 macros, 0 operations, 0 seed files, 1 source, 0 exposures, 0 metrics
17:13:20  
17:13:21  Concurrency: 1 threads (target='dev')
17:13:21  
17:13:21  1 of 1 START view model 00dev.job_posts ........................................ [RUN]
17:13:23  1 of 1 OK created view model 00dev.job_posts ................................... [OK in 1.52s]
17:13:23  
17:13:23  Finished running 1 view model in 2.91s.
17:13:23  
17:13:23  Completed successfully
17:13:23  
17:13:23  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1
/home/butch2/play/experiments/hn_whos_hiring/notebooks
